<a href="https://colab.research.google.com/github/rajeevfromkrec/pytorch/blob/master/text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torchtext==0.5




     |████████████████████████████████| 81kB 2.1MB/s 
     |████████████████████████████████| 1.1MB 7.1MB/s 
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [2]:
import torch
import torchtext
from torchtext.datasets import text_classification
import os
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,random_split
import time
import re
from torchtext.data.utils import ngrams_iterator
from torchtext.data.utils import get_tokenizer

In [3]:
NGRAMS=2
batch_size=16

In [4]:
!ls
!mkdir data


sample_data


In [5]:
!ls

data  sample_data


In [6]:
train_dataset, test_dataset= text_classification.DATASETS['DBpedia'](root='./data',ngrams=NGRAMS,vocab=None)

dbpedia_csv.tar.gz: 68.3MB [00:01, 57.9MB/s]
560000lines [00:53, 10563.09lines/s]
560000lines [01:48, 5153.09lines/s]
70000lines [00:12, 5452.68lines/s]


In [7]:
print( len(train_dataset))
print(len(train_dataset.get_labels()))
print( len(test_dataset))
print(len(test_dataset.get_labels()))

560000
14
70000
14


In [8]:
device =torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
class textSemntiment(nn.Module):
  def __init__(self,vocab_size,embedding_dim,num_classes):
    super().__init__()
    self.embedding=nn.EmbeddingBag(vocab_size,embedding_dim,sparse=True)
    self.fc=nn.Linear(embedding_dim,num_classes)
    self.init_weights()

  def init_weights(self):
    initrange = 0.5
    self.embedding.weight.data.uniform_(-initrange, initrange)
    self.fc.weight.data.uniform_(-initrange, initrange)
    self.fc.bias.data.zero_()

  def forward(self,text,offsets):
    embedded=self.embedding(text,offsets)
    out=self.fc(embedded)
    return out



In [11]:
vocab_size=len(train_dataset.get_vocab())
embedding_dim=32
num_classes=len(train_dataset.get_labels())
model=textSemntiment(vocab_size=vocab_size,embedding_dim=embedding_dim,num_classes=num_classes)

In [12]:
print(model)

textSemntiment(
  (embedding): EmbeddingBag(6375026, 32, mode=mean)
  (fc): Linear(in_features=32, out_features=14, bias=True)
)


In [34]:
text=train_dataset[55000][1]
#print([0]+[len(text)])

offset=[[0,67],[0,89],[0,43],[0,55]]
print(offset)
offsets=torch.tensor(offset[:-1]).cumsum(dim=0)
print(offsets)

[[0, 67], [0, 89], [0, 43], [0, 55]]
tensor([[  0,  67],
        [  0, 156],
        [  0, 199]])


In [40]:
def generate_batch(batch):
    label = torch.tensor([entry[0] for entry in batch])
    text = [entry[1] for entry in batch]
    offsets = [0] + [len(entry) for entry in text]
    # torch.Tensor.cumsum returns the cumulative sum
    # of elements in the dimension dim.
    # torch.Tensor([1.0, 2.0, 3.0]).cumsum(dim=0)

    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text = torch.cat(text)
    return text, offsets, label

In [ ]:
from torch.utils.data import DataLoader

In [42]:
def train_func(sub_train_):
  train_loss=0.0
  train_acc=0.0
  data=DataLoader(sub_train_,batch_size=batch_size, shuffle=True,collate_fn=generate_batch)

  for i,(text,offset,cls) in enumerate(data):
    optimizer.zero_grad()
    text,offset,cls=text.to(device),offset.to(device),cls.to(device)
    output=model(text,offset)
    loss=criterion(output,cls)
    train_loss  +=loss.item()
    loss.backward()
    optimizer.step()
    train_acc +=(output.argmax(1)==cls).sum().item()
  scheduler.step()
  return train_loss/len(sub_train_),train_acc/len(sub_train_)


In [45]:
def test(data_):
    loss = 0
    acc = 0
    data = DataLoader(data_, batch_size=batch_size, collate_fn=generate_batch)
    for text, offsets, cls in data:
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        with torch.no_grad():
            output = model(text, offsets)
            loss = criterion(output, cls)
            loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()

    return loss / len(data_), acc / len(data_)

In [ ]:
n_epochs=5
min_valid_loss=float('inf')
criterion=torch.nn.CrossEntropyLoss().to(device)
optimizer=torch.optim.SGD(model.parameters(),lr=4.0)
scheduler=torch.optim.lr_scheduler.StepLR(optimizer,1,gamma=0.9)
train_len = int(len(train_dataset) * 0.95)
sub_train_, sub_valid_ = \
random_split(train_dataset, [train_len, len(train_dataset) - train_len])

for epoch in range(n_epochs):

    start_time = time.time()
    train_loss, train_acc = train_func(sub_train_)
    valid_loss, valid_acc = test(sub_valid_)

    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

Epoch: 1  | time in 2 minutes, 0 seconds
	Loss: 0.0025(train)	|	Acc: 98.9%(train)
	Loss: 0.0000(valid)	|	Acc: 99.1%(valid)
Epoch: 2  | time in 2 minutes, 1 seconds
	Loss: 0.0012(train)	|	Acc: 99.5%(train)
	Loss: 0.0000(valid)	|	Acc: 99.1%(valid)
Epoch: 3  | time in 2 minutes, 0 seconds
	Loss: 0.0006(train)	|	Acc: 99.8%(train)
	Loss: 0.0000(valid)	|	Acc: 99.1%(valid)
